<a href="https://colab.research.google.com/github/aicuai/coloso/blob/main/AICU_Coloso1_Original_Tool.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 超入門: Stable Diffusionではじめる画像生成AI

## Section 04. Stability AI API の革新的機能

### このGoogle Colabノートブックへの短縮URL https://j.aicu.ai/c1s4

こちらは Coloso受講生のための特別なコンテンツになっております。
内容に関する質問は Coloso 当該コンテンツの質問ページにてお問い合わせください。



Section 01.　画像生成AIの歴史と現在、今後の価値。


Section 02. 　環境構築


Section 03.　基礎編[SD3時代のモデル評価とプロンプト探求]


**Section 04. 　Stability AI API の革新的機能**


Section 05.　応用編 [バーチャルファッションモデルの開発]


Section 06.　活用編 [広告バリエーションの展開と評価手法]



Section 07.　法律・倫理・未来



# 使用できる機能

# Generate

*  Stable Image Ultra

# Upscale 高解像度化

*  Creative
*  Conservative

# Edit 画像編集機能

*  Search and Replace
*  Outpaint
*  Remove Background


# Control

*  Structure
*  Style


https://github.com/Stability-AI/stability-sdk/tree/main/nbs


[Stability AI 公式サンプル](https://j.aicu.ai/SAIci)との違いは以下の点です。

- APIキーを左側のセキュリティ機能で設定します。設定がない場合はパスワード方式で入力します
- 生成された画像を MyDrive/AICU-Coloso/ に保存します
- 与えたプロンプトやシードをファイル名に残します





In [ ]:
#@markdown # セットアップ
from io import BytesIO
import IPython
import json
import os
from PIL import Image
import requests
import time
from google.colab import output, userdata, drive
import getpass
from datetime import datetime
import re
import base64

#@markdown ## Stability AI の APIキーを設定

def get_stability_key():
    try:
        stability_key = userdata.get('STABILITY_KEY')
        if stability_key is None:
            raise ValueError("STABILITY_KEY not found in userdata")
        print("STABILITY_KEY が設定されました")
        return stability_key
    except Exception as e:
        print("Could not retrieve STABILITY_KEY from userdata. Please enter it manually.")
        return getpass.getpass('Enter your API Key')

# To get your API key visit https://platform.stability.ai/account/keys
STABILITY_KEY = get_stability_key()

#@markdown ## Google Drive のマウントと設定
drive.mount('/content/drive')

ProjectPath = "/content/drive/MyDrive/AICU-Coloso" #@param {type:"string"}
MAX_PROMPT_LENGTH = 100  #@param {type:"integer"} #プロンプトの最大文字数

# 作業ディレクトリの確認と作成
if not os.path.exists(ProjectPath):
    os.makedirs(ProjectPath)
    print(f"作業ディレクトリを作成しました: {ProjectPath}")
else:
    print(f"作業ディレクトリが存在します: {ProjectPath}")

def test_api_connection():
    url = "https://api.stability.ai/v1/user/account"
    headers = {
        "Authorization": f"Bearer {STABILITY_KEY}"
    }
    try:
        response = requests.get(url, headers=headers)
        if response.status_code == 200:
            print("APIへの接続テストに成功しました")
            return True
        else:
            print(f"APIへの接続テストに失敗しました。ステータスコード: {response.status_code}")
            return False
    except Exception as e:
        print(f"APIへの接続テスト中にエラーが発生しました: {str(e)}")
        return False

def send_generation_request(engine_id, params):
    url = f"https://api.stability.ai/v1/generation/{engine_id}/text-to-image"
    headers = {
        "Accept": "application/json",
        "Authorization": f"Bearer {STABILITY_KEY}"
    }

    response = requests.post(url, headers=headers, json=params)
    if response.status_code != 200:
        raise Exception(f"Non-200 response: {response.text}")

    data = response.json()
    return data

def save_result_image(params, output_image):
    max_prompt_length = MAX_PROMPT_LENGTH
    seed = params.get('seed', '')
    prompt = params.get('prompt', '')
    negative_prompt = params.get('negative_prompt', '')

    # ファイル名に使用できない文字を置換
    def clean_string(s):
        return re.sub(r'[^\w\-_\. ]', '_', s)

    prompt = clean_string(prompt)[:max_prompt_length]
    negative_prompt = clean_string(negative_prompt)[:max_prompt_length]

    base_filename = f"S{seed}_PP_{prompt}_NP{negative_prompt}"

    num = 0
    while True:
        filename = f"{base_filename}_{num}.png" if num > 0 else f"{base_filename}.png"
        full_path = os.path.join(ProjectPath, filename)
        if not os.path.exists(full_path):
            break
        num += 1

    with open(full_path, "wb") as f:
        f.write(output_image)

    print(f"Saved image: {full_path}")
    return full_path

# STABILITY_KEYの設定確認とAPIへのテスト送信
if STABILITY_KEY:
    test_api_connection()
else:
    print("STABILITY_KEY が設定されていません")
#@title Define functions

def send_generation_request(
    host,
    params,
):
    headers = {
        "Accept": "image/*",
        "Authorization": f"Bearer {STABILITY_KEY}"
    }

    # Encode parameters
    files = {}
    image = params.pop("image", None)
    mask = params.pop("mask", None)
    if image is not None and image != '':
        files["image"] = open(image, 'rb')
    if mask is not None and mask != '':
        files["mask"] = open(mask, 'rb')
    if len(files)==0:
        files["none"] = ''

    # Send request
    print(f"Sending REST request to {host}...")
    response = requests.post(
        host,
        headers=headers,
        files=files,
        data=params
    )
    if not response.ok:
        raise Exception(f"HTTP {response.status_code}: {response.text}")

    return response

def send_async_generation_request(
    host,
    params,
):
    headers = {
        "Accept": "application/json",
        "Authorization": f"Bearer {STABILITY_KEY}"
    }

    # Encode parameters
    files = {}
    if "image" in params:
        image = params.pop("image")
        files = {"image": open(image, 'rb')}

    # Send request
    print(f"Sending REST request to {host}...")
    response = requests.post(
        host,
        headers=headers,
        files=files,
        data=params
    )
    if not response.ok:
        raise Exception(f"HTTP {response.status_code}: {response.text}")

    # Process async response
    response_dict = json.loads(response.text)
    generation_id = response_dict.get("id", None)
    assert generation_id is not None, "Expected id in response"

    # Loop until result or timeout
    timeout = int(os.getenv("WORKER_TIMEOUT", 500))
    start = time.time()
    status_code = 202
    while status_code == 202:
        response = requests.get(
            f"{host}/result/{generation_id}",
            headers={
                **headers,
                "Accept": "image/*"
            },
        )

        if not response.ok:
            raise Exception(f"HTTP {response.status_code}: {response.text}")
        status_code = response.status_code
        time.sleep(10)
        if time.time() - start > timeout:
            raise Exception(f"Timeout after {timeout} seconds")

    return response

In [ ]:
#@title Stable Image Ultra

#@markdown ## プロンプトを3分割して入力できます
prompt1 = "photo realistic, full body shot of 1girl,  " #@param {type:"string"}
prompt2 = "zoom out the camera, walking, animal ears, " #@param {type:"string"}
prompt3 = "solo, brown hair, overalls, dark skin, cat ears, short hair, dark-skinned female, simple background, choker, brown eyes, sweater, pink choker, long sleeves, closed mouth, yellow shirt, white background, bright pupils, white pupils, pink basketball shoes" #@param {type:"string"}
prompt = prompt1 + prompt2 + prompt3

negative_prompt = "" #@param {type:"string"}
aspect_ratio = "21:9" #@param ["21:9", "16:9", "3:2", "5:4", "1:1", "4:5", "2:3", "9:16", "9:21"]
seed = 39 #@param {type:"integer"}
output_format = "png" #@param ["webp", "jpeg", "png"]

host = f"https://api.stability.ai/v2beta/stable-image/generate/ultra"

params = {
    "prompt" : prompt,
    "negative_prompt" : negative_prompt,
    "aspect_ratio" : aspect_ratio,
    "seed" : seed,
    "output_format": output_format
}

response = send_generation_request(
    host,
    params
)

# Decode response
output_image = response.content
finish_reason = response.headers.get("finish-reason")
seed = response.headers.get("seed")

# Check for NSFW classification
if finish_reason == 'CONTENT_FILTERED':
    raise Warning("Generation failed NSFW classifier")

# Save result using save_result_image function
saved_path = save_result_image(params, output_image)

print(f"Saved image: {saved_path}")

output.no_vertical_scroll()
print("Result image:")
IPython.display.display(Image.open(saved_path))

photo realistic, full body shot of 1girl,  zoom out the camera, walking, animal ears, solo, brown hair, overalls, dark skin, cat ears, short hair, dark-skinned female, simple background, choker, brown eyes, sweater, pink choker, long sleeves, closed mouth, yellow shirt, white background, bright pupils, white pupils, pink basketball shoes

 - anime screencap, a young woman, long hair, black hair, red dress, evening dress, standing, full body, in a sophisticated setting,

 - text, lowres, bad anatomy, bad hands, error, missing fingers, (realistic, nose, lipstick:1.0), (abs, muscular, rib:1.0)

In [ ]:
#@title Style

#@markdown - 画像を左のファイル・フォルダにドラッグ・アンド・ドロップする
#@markdown - 右クリックして「パスをコピー」を選択する
#@markdown - そのパスを下の画像フィールドに貼り付ける
#@markdown <br><br>

image = "/content/drive/MyDrive/AICU-Coloso/test.png" #@param {type:"string"}
prompt = "a photo of a girl, solo, sitting, cross her legs, window, socks, moon, long hair, skirt, paper, plant, chair, blue theme, kneehighs, holding, long sleeves, from the side, holding paper, shoes, shirt, potted plant, dutch angle, black socks, black footwear, full moon, indoors, red moon, black skirt, pleated skirt, school uniform, reflection, white shirt" #@param {type:"string"}
negative_prompt = "" #@param {type:"string"}
aspect_ratio = "16:9" #@param ["21:9", "16:9", "3:2", "5:4", "1:1", "4:5", "2:3", "9:16", "9:21"]
fidelity = 0.3  #@param {type:"slider", min:0, max:1, step:0.05}
start_seed = 0 #@param {type:"integer"}
end_seed = 10 #@param {type:"integer"}
output_format = "png" #@param ["webp", "jpeg", "png"]

host = f"https://api.stability.ai/v2beta/stable-image/control/style"

for seed in range(start_seed, end_seed + 1):
    params = {
        "fidelity" : fidelity,
        "image" : image,
        "seed" : seed,
        "output_format": output_format,
        "prompt" : prompt,
        "negative_prompt" : negative_prompt,
    }

    response = send_generation_request(
        host,
        params
    )

    # Decode response
    output_image = response.content
    finish_reason = response.headers.get("finish-reason")
    seed = response.headers.get("seed")

    # Check for NSFW classification
    if finish_reason == 'CONTENT_FILTERED':
        print(f"Generation failed NSFW classifier for seed {seed}")
        continue

    # Save and display result
    saved_path = save_result_image(params, output_image)

    print(f"Saved image: {saved_path}")

    output.no_vertical_scroll()
    print(f"Result image for seed {seed}:")
    IPython.display.display(Image.open(saved_path))





print("Original image:")
IPython.display.display(Image.open(image))


In [ ]:
#@title Creative Upscaler

#@markdown - 画像を左のファイル・フォルダにドラッグ・アンド・ドロップする
#@markdown - 右クリックして「パスをコピー」を選択する
#@markdown - そのパスを下の画像フィールドに貼り付ける
#@markdown <br><br>

image = "/content/edited_o_ob_You_can_understand_it_better_by_creating_it_from_scratch_L_a3ca53cb-230f-4471-b6e3-9546a6ea02fa_8.png" #@param {type:"string"}
prompt = "vivid color" #@param {type:"string"}
negative_prompt = "" #@param {type:"string"}
seed = 8 #@param {type:"integer"}
creativity = 0.30 #@param {type:"number"}
output_format = "png" #@param ["webp", "jpeg", "png"]

host = f"https://api.stability.ai/v2beta/stable-image/upscale/creative"

params = {
    "prompt" : prompt,
    "negative_prompt" : negative_prompt,
    "seed" : seed,
    "creativity" : creativity,
    "image" : image,
    "output_format": output_format
}

response = send_async_generation_request(
    host,
    params
)

# Decode response
output_image = response.content
finish_reason = response.headers.get("finish-reason")
seed = response.headers.get("seed")

# Check for NSFW classification
if finish_reason == 'CONTENT_FILTERED':
    raise Warning("Generation failed NSFW classifier")


# Save and display result
saved_path = save_result_image(params, output_image)

print(f"Saved image: {saved_path}")


print("Original image:")
IPython.display.display(Image.open(image))
print("Result image:")
IPython.display.display(Image.open(saved_path))

In [ ]:
#@title Conservative Upscaler

#@markdown - 画像を左のファイル・フォルダにドラッグ・アンド・ドロップする
#@markdown - 右クリックして「パスをコピー」を選択する
#@markdown - そのパスを下の画像フィールドに貼り付ける
#@markdown <br><br>

image = "/content/edited_o_ob_You_can_understand_it_better_by_creating_it_from_scratch_L_a3ca53cb-230f-4471-b6e3-9546a6ea02fa_8.png" #@param {type:"string"}
prompt = "a photo of a girl, solo, sitting, cross her legs, window, socks, moon, long hair, skirt, paper, plant, chair, blue theme, kneehighs, holding, long sleeves, from the side, holding paper, shoes, shirt, potted plant, dutch angle, black socks, black footwear, full moon, indoors, red moon, black skirt, pleated skirt, school uniform, reflection, white shirt" #@param {type:"string"}
negative_prompt = "" #@param {type:"string"}
seed = 8 #@param {type:"integer"}
creativity = 0.35 #@param {type:"number"}
output_format = "png" #@param ["webp", "jpeg", "png"]

host = f"https://api.stability.ai/v2beta/stable-image/upscale/conservative"

params = {
    "prompt" : prompt,
    "negative_prompt" : negative_prompt,
    "seed" : seed,
    "creativity" : creativity,
    "image" : image,
    "output_format": output_format
}

response = send_generation_request(
    host,
    params
)

# Decode response
output_image = response.content
finish_reason = response.headers.get("finish-reason")
seed = response.headers.get("seed")

# Check for NSFW classification
if finish_reason == 'CONTENT_FILTERED':
    raise Warning("Generation failed NSFW classifier")


# Save and display result
saved_path = save_result_image(params, output_image)
print(f"Saved image: {saved_path}")



print("Original image:")
IPython.display.display(Image.open(image))
print("Result image:")
IPython.display.display(Image.open(saved_path))

In [ ]:
#@title Structure

#@markdown - 画像を左のファイル・フォルダにドラッグ・アンド・ドロップする
#@markdown - 右クリックして「パスをコピー」を選択する
#@markdown - そのパスを下の画像フィールドに貼り付ける
#@markdown <br><br>

image = "/content/drive/MyDrive/MyArts/TwinInMarche.png" #@param {type:"string"}

prompt_part1 = "photorealistic, multiple girls, face focus, looking each other, blue eyes, brown hair, earrings, back pack, short hair, long hair, blush, shelf, hair ornament, brown eyes, summer, sleeveless, bangs, backpack, upper body,"  #@param {type:"string"}
prompt_part2 = "freckles, smile showing teeth, snub nose, half eye closed, blushing, " #@param {type:"string"}
prompt_part3 = "indoors, bedroom, flower in the back, bokeh, eye catch lighting, soft focus, blur, " #@param {type:"string"}
prompt = prompt_part1 + prompt_part2 + prompt_part3


# prompt = "otome illustration of a 20 years old young man with red hair wearing male yukata, smiling and reaching out his hand towards the viewer, the camera from his back, " #@param {type:"string"}
negative_prompt = "text, low quality, distorted, ugly, deformed, poorly drawn, wrong anatomy, bad proportions, bad composition, no text, bad hands, bad arms" #@param {type:"string"}
control_strength = 0.75  #@param {type:"slider", min:0, max:1, step:0.05}
seed = 10 #@param {type:"integer"}
output_format = "png" #@param ["webp", "jpeg", "png"]
path = "/content/drive/MyDrive/MyArts/" #@param {type:"string"}

host = f"https://api.stability.ai/v2beta/stable-image/control/structure"

params = {
    "control_strength" : control_strength,
    "image" : image,
    "seed" : seed,
    "output_format": output_format,
    "prompt" : prompt,
    "negative_prompt" : negative_prompt,
}

response = send_generation_request(
    host,
    params
)

# Decode response
output_image = response.content
finish_reason = response.headers.get("finish-reason")
seed = response.headers.get("seed")

# Check for NSFW classification
if finish_reason == 'CONTENT_FILTERED':
    raise Warning("Generation failed NSFW classifier")



# Save and display result
saved_path = save_result_image(params, output_image)

print(f"Saved image: {saved_path}")


output.no_vertical_scroll()
print("Result image:")
IPython.display.display(Image.open(saved_path))



https://colab.research.google.com/drive/1_-YCrdQfW5HGvMqjwGfFIYuXYlKUrZm1#scrollTo=oVaB2KLvDnNm

In [ ]:
#@title Remove Background

#@markdown - 画像を左のファイル・フォルダにドラッグ・アンド・ドロップする
#@markdown - 右クリックして「パスをコピー」を選択する
#@markdown - そのパスを下の画像フィールドに貼り付ける
#@markdown <br><br>


image = "/content/drive/MyDrive/MyArts/generated_39.png" #@param {type:"string"}
output_format = "png" #@param ["webp", "jpeg", "png"]

host = f"https://api.stability.ai/v2beta/stable-image/edit/remove-background"

params = {
    "image" : image,
    "output_format": output_format
}

response = send_generation_request(
    host,
    params
)

# Decode response
output_image = response.content
finish_reason = response.headers.get("finish-reason")
seed = response.headers.get("seed")

# Check for NSFW classification
if finish_reason == 'CONTENT_FILTERED':
    raise Warning("Generation failed NSFW classifier")



# Save and display result
saved_path = save_result_image(params, output_image)

print(f"Saved image: {saved_path}")


output.no_vertical_scroll()
print("Original image:")
IPython.display.display(Image.open(image))
print("Result image:")
IPython.display.display(Image.open(saved_path))